In [1]:
from transformers import AutoProcessor, AutoModel
from decord import VideoReader, cpu
from IPython.display import HTML

from utils import compute_clip_sim, get_samples

/disk/users/zwu/envs/xclip/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = input() or "microsoft/xclip-base-patch16-zero-shot"
processor, model = AutoProcessor.from_pretrained(model_name), AutoModel.from_pretrained(model_name)

In [4]:
filename = input()
vr = VideoReader(filename, ctx=cpu(0))
HTML(f'<video controls> <source src="{filename}" type="video/mp4"> </video>')

 videos/tank inside.mp4


In [12]:
classes = ["tank movement", "office laptop", "moving cars", 'aircraft in hangar', 'machine']
classes = ["inside of a military vehicle", "sports car engine", "chemistry experiment", 'missile launcher', 'aircraft in hangar']

In [13]:
frames = get_samples(vr)
video_features = model.get_video_features(**processor(videos=list(frames), return_tensors="pt"))
text_features = model.get_text_features(**processor(text=classes, return_tensors="pt", padding=True))

Reducing sample interval to 1
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [14]:
logits = compute_clip_sim(video_features, text_features)
*logits.T, classes[logits.argmax()]

(tensor([0.2393, 0.2336, 0.2255, 0.2205, 0.2067]),
 'inside of a military vehicle')